As the number of groups increases, which algorithm performs better at scaling?

In [72]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Step 1: Load the data
df = pd.read_parquet(
    "results/test.parquet", 
    columns=["dist", "algorithm", "n_groups", "np", "value"],
    filters=[("n_rows", "==", 8_000_000), ("attribute", "==", "aggregation_time")],
)

# Step 2: Preprocess
df["value"] = df["value"].apply(lambda x: float(x.removesuffix("ms")))
df = df.groupby(["dist", "algorithm", "n_groups", "np"])["value"].mean().reset_index()
df = df.rename(columns={"value": "latency"})

# Step 3: Add Speedup Column
baseline = df[df["np"] == 1][["dist", "algorithm", "n_groups", "latency"]]
baseline = baseline.rename(columns={"latency": "baseline_latency"})
df = df.merge(baseline, on=["dist", "algorithm", "n_groups"], how="left")
df["speedup"] = df["baseline_latency"] / df["latency"]
df = df.drop(columns=["baseline_latency"])

# Step 4: Add Tolerance Column
# Find the latency at the minimal n_groups for each (dist, algorithm, np)
min_group_latency = (
    df.loc[df.groupby(["dist", "algorithm", "np"])["n_groups"].idxmin()]
    [["dist", "algorithm", "np", "latency"]]
    .rename(columns={"latency": "min_group_latency"})
)

df = df.merge(min_group_latency, on=["dist", "algorithm", "np"], how="left")
df["tolerance"] = df["latency"] / df["min_group_latency"]
df = df.drop(columns=["min_group_latency"])

# Step 5: Display
for algorithm in df["algorithm"].unique():
    display(df[(df["np"] == df["np"].max()) & (df["algorithm"] == algorithm)])

,dist,algorithm,n_groups,np,latency,speedup,tolerance
4,uniform,duckdbish-two-phase,20000,16,56.8,9.542254,1.000000
9,uniform,duckdbish-two-phase,200000,16,290.6,2.485203,5.116197
14,uniform,duckdbish-two-phase,2000000,16,731.0,2.717100,12.869718


,dist,algorithm,n_groups,np,latency,speedup,tolerance
19,uniform,implicit-repartitioning,20000,16,97.8,5.599182,1.000000
24,uniform,implicit-repartitioning,200000,16,109.4,6.482633,1.118609
29,uniform,implicit-repartitioning,2000000,16,209.8,8.212583,2.145194


,dist,algorithm,n_groups,np,latency,speedup,tolerance
34,uniform,three-phase-radix,20000,16,56.0,9.639286,1.000000
39,uniform,three-phase-radix,200000,16,289.8,2.536232,5.175000
44,uniform,three-phase-radix,2000000,16,724.0,2.481768,12.928571


,dist,algorithm,n_groups,np,latency,speedup,tolerance
49,uniform,two-phase-central-merge,20000,16,67.4,7.735905,1.000000
54,uniform,two-phase-central-merge,200000,16,437.8,1.767017,6.495549
59,uniform,two-phase-central-merge,2000000,16,1989.6,0.893245,29.519288


,dist,algorithm,n_groups,np,latency,speedup,tolerance
64,uniform,two-phase-radix,20000,16,56.6,9.699647,1.000000
69,uniform,two-phase-radix,200000,16,248.6,2.943685,4.392226
74,uniform,two-phase-radix,2000000,16,468.2,4.382742,8.272085
